In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob

#filename queue
filename_queue=glob.glob('./rawdata/data[1-4].csv')


In [2]:
#showing shape of the data
for file in filename_queue:
    df=pd.read_csv(file)
    print(file,df.shape)
#    for row in df:
#        print(row.shape())

./rawdata/data2.csv (36, 3604)
./rawdata/data3.csv (199, 5)
./rawdata/data1.csv (48, 3604)
./rawdata/data4.csv (9, 5)


In [3]:
#Create return path
return_path='./parsed/data.csv'
create_file=open(return_path,'w')

In [4]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
label_count=np.zeros(16)

return_file=open(return_path,'a')
writer=csv.writer(return_file)
writer.writerow(["labl","X_acc","Y_acc","Z_acc","X_gyro","Y_gyro","Z_gyro"])

for filename in filename_queue:
    file=open(filename, newline='')

    reader=csv.reader(file)
    header=next(reader)
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        label_count[labl]+=1
        
        
        X_acc=(row[4:604])
        Y_acc=(row[604:1204])
        Z_acc=(row[1204:1804])
        X_gyro=(row[1804:2404])
        Y_gyro=(row[2404:3004])
        Z_gyro=(row[3004:3604])

        #6*6*100 float array : will be feature-extracted
        window=np.array([[X_acc[0:100],X_acc[100:200],X_acc[200:300],X_acc[300:400],X_acc[400:500],X_acc[500:600]],
                [Y_acc[0:100],Y_acc[100:200],Y_acc[200:300],Y_acc[300:400],Y_acc[400:500],Y_acc[500:600]],
                [Z_acc[0:100],Z_acc[100:200],Z_acc[200:300],Z_acc[300:400],Z_acc[400:500],Z_acc[500:600]],
                [X_gyro[0:100],X_gyro[100:200],X_gyro[200:300],X_gyro[300:400],X_gyro[400:500],X_gyro[500:600]],
                [Y_gyro[0:100],Y_gyro[100:200],Y_gyro[200:300],Y_gyro[300:400],Y_gyro[400:500],Y_gyro[500:600]],
                [Z_gyro[0:100],Z_gyro[100:200],Z_gyro[200:300],Z_gyro[300:400],Z_gyro[400:500],Z_gyro[500:600]]])
    
        #5 features  what else?2
        window_mean=window.mean(axis=-1)
        window_stddev=window.std(axis=-1)
        window_median=np.median(window,axis=-1)
        window_percent25=np.percentile(window,25,axis=-1)
        window_percent75=np.percentile(window,75,axis=-1)

        window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
        window_feature=window_feature.reshape(180)
    
        if (label_count[labl]%5!=1):
            training_set.append(window_feature)
            training_label.append(labl)
        else:
            test_set.append(window_feature)
            test_label.append(labl)
        

In [5]:
#print(window_feature.shape)
#print(training_label)
print(test_set)

[array([-1.14355386e+00, -7.24714160e-01,  4.02927380e-01, -3.51287896e+00,
       -1.47756798e+00, -2.55673090e-01, -4.71792810e-01, -2.83719372e+00,
       -5.75885406e+00, -7.99686101e+00, -1.50293190e-01, -1.32476236e+00,
        9.87931860e+00,  8.55192418e+00,  4.76442096e+00,  2.52306399e+00,
        1.03034225e+01,  9.64001247e+00,  2.74189996e+00, -9.23650010e+00,
       -9.73917005e+01,  7.87479005e+01,  2.27808002e+01, -4.27210008e+00,
       -5.30950004e+00, -2.82093000e+01, -4.60879985e+00,  1.30767001e+01,
       -1.35569000e+01, -5.04000012e+00,  1.82280000e+00,  3.24198002e+01,
        7.61039999e+00, -1.34897001e+01, -1.06631001e+01, -2.37965000e+01,
        9.73920854e-01,  1.98853174e+00,  2.33671931e+00,  4.41532761e-01,
        1.85095403e+00,  1.47029550e+00,  1.19072829e+00,  3.71818667e+00,
        4.51011083e+00,  1.09116262e+00,  1.37023793e+00,  9.77758998e-01,
        8.30843214e-01,  2.30178859e+00,  4.91085400e+00,  2.23910404e+00,
        9.36922539e-01, 

In [6]:
#SVM
#prepare the data
X=training_set
y=training_label
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)

clf.fit(X,y)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

test_pred1=clf.predict(test_set)

ohc=OneHotEncoder(categories=[range(16)])
onehot_pred1=test_pred1.reshape(-1,1)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
print(test_pred1)
#print(onehot_pred1)

#print(accuracy_score(test_pred1,test_label))
#print(clf.n_support_)
#confusion_matrix(test_pred1, test_label)

[12  9 10  9  9 12 10 10  9  6  7  1  6  6  0 12  6  5  7 12 12  6 12  1
 12 12 12 10 12 12 12  5  0 12 12  6  6  5  5  0  1 12  6  1  5  1 10 12
 13  8 12  8  5 13  0 12  5  8  6 12  1 12  5  5 12  6]


In [7]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
clf.fit(X, y)
test_pred2=clf.predict(test_set)
print(accuracy_score(test_pred2,test_label))

onehot_pred2=test_pred2.reshape(-1,1)
onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

0.6212121212121212


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier()
clf.fit(X, y)
test_pred3=clf.predict(test_set)
print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
onehot_pred3=test_pred2.reshape(-1,1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
#print(onehot_pred3)

0.7878787878787878


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
#ensemble 1 : unanimous consensus
ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
print(ensemble1_pred)

[ 0  9 10  9  9  0  0 10  9  6  7  1  6  6  0 12  6  5  7 12  0  6 12  1
  0 12  0 10  0  0 12  5  0  0 12  6  0  0  5  0  1 12  6  1  5  0  0  0
 13  8  0  8  0 13  0 12  5  8  6 12  1  0  0  0  0  6]


In [10]:
#ensemble 2 : majority consensus
ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
ensemble2_pred= ensemble2_pred.astype('int64') 
ensemble2_pred= ensemble2_pred.astype('float64') 
ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
print(ensemble2_pred)
print(test_label)
print(accuracy_score(ensemble2_pred,test_label))


[ 7  9 10  9  9  7  9 10  9  6  7  1  6  6  0 12  6  5  7 12  9  6 12  1
 10 12  2 10  9  9 12  5  0  3 12  6  9  7  5  7  1 12  6  1  5 14  3 14
 13  8 14  8 14 13  0 12  5  8  6 12  1 14  1 14  9  6]
[12, 9, 10, 10, 9, 12, 10, 10, 9, 6, 7, 1, 6, 2, 0, 0, 6, 5, 7, 3, 4, 6, 12, 1, 12, 12, 12, 0, 9, 12, 6, 5, 0, 12, 12, 6, 7, 2, 5, 0, 3, 12, 6, 1, 5, 1, 10, 13, 13, 8, 14, 8, 14, 13, 0, 12, 5, 8, 6, 12, 1, 5, 8, 3, 12, 6]
0.6212121212121212
